In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

In [4]:
from enum import Enum
class Labels(Enum):
    # Normal behaviors
    STD = 1
    WAL = 2
    JOG = 3
    JUM = 4
    STU = 5
    STN = 6
    SCH = 7
    SIT = 8
    CHU = 9
    SCI = 10
    CSO = 11
    LYI = 12
    # Falls
    FOL = 13
    FKL = 14
    BSC = 15
    SDL = 16

In [5]:
# signal magnitude vector
def smv(accX:float, accY:float, accZ:float)->float:
    return math.sqrt(pow(accX,2)+pow(accY,2)+pow(accZ,2))

In [8]:
class Window:
    def __init__(self, raw_df, label):
        self.raw_df = raw_df
        self.label = label
        dict_feature = Window.extract(raw_df)
        
        self.smv_list = dict_feature['smv_list']
        self.mean_smv = dict_feature['mean_smv']
        self.max_smv = dict_feature['max_smv']
        self.min_smv = dict_feature['min_smv']
        self.duration = dict_feature['duration']

    @staticmethod
    def extract(df:pd.DataFrame):
        smv_result = []
        for line in range(0, df.shape[0]):
            acc_x = df['acc_x'][line]
            acc_y = df['acc_y'][line]
            acc_z = df['acc_z'][line]
            smv_result.append(smv(acc_x, acc_y, acc_z))
            
#             gryo_x = df['gryo_x'][line]
#             gryo_y = df['gryo_y'][line]
#             gryo_z = df['gryo_z'][line]
            
        smv_list = np.asarray(smv_result)
        mean_smv = np.mean(smv_list)
        max_smv = np.max(smv_list)
        min_smv = np.min(smv_list)
        duration = np.argmin(smv_list) - np.argmax(smv_list)
        return {'smv_list':smv_list, 'mean_smv':mean_smv, 'max_smv':max_smv, 'min_smv':min_smv, 'duration':duration}      
    
    @staticmethod
    def getWindows(FilePath:str, MainEvent:str, SamplingRate: int, OverlapRate:float, WindowSize:float):
        window_frames = SamplingRate * WindowSize
        interval_frames = window_frames * (1 - OverlapRate)
        windows = []
        raw_data = pd.read_csv(FilePath)
        if MainEvent in ['FOL', 'FKL', 'BSC', 'SDL']:
            SMVs = np.array([])
            for line in range(0, raw_data.shape[0]):
                acc_x = raw_data['acc_x'][line]
                acc_y = raw_data['acc_y'][line]
                acc_z = raw_data['acc_z'][line]
                SMVs = np.append(SMVs, smv(acc_x, acc_y, acc_z))
            max_smv_frame = np.argmax(SMVs)
            min_smv_frame = np.argmax(SMVs)
            isFall = True
        else:
            isFall = False
        # total number of lines in the raw-data csv file
        total_frames = raw_data.shape[0]
        num_of_windows = math.ceil((total_frames - window_frames) / interval_frames) + 1
        for i in range(num_of_windows):
            start_iloc = i * interval_frames
            end_iloc = i * interval_frames + window_frames - 1
            if end_iloc > total_frames:
                end_iloc =  total_frames
            window_df = raw_data.loc[start_iloc : end_iloc, ['acc_x', 'acc_y', 'acc_z', 
                                      'gyro_x', 'gyro_y', 'gyro_z', 'label']]
            # class label : 0 for normal behaviors / 1 for falls
            if not isFall:
                window_label = 0
            else:
                if start_iloc <= max_smv_frame and min_smv_frame <= end_iloc:
                    window_label = 1 
                else:
                    window_label = 0       
            window_df = window_df.reset_index(drop=True)
            newWindow = Window(window_df, window_label)
            windows.append(newWindow)
        
        return windows

    def visualize(self):
        Xs = np.arange(0,len(self.smv_list))
        plt.plot(Xs, self.smv_list)
        plt.ylim(ymin=0, ymax=35)
        if self.label == 0:
            plt.title('Normal behavior')
        else:
            plt.title('Fall event')
        plt.xlabel('Sample Reading Number')
        plt.ylabel('Signal Matitude Vector m/$\mathregular{s^2}$')
        plt.show()
        
    def export(self):
#         if self.label == 0:
#             self.smv_list.to_csv(SavePath1)      
        newLine = pd.DataFrame([[self.mean_smv,self.max_smv,self.min_smv,self.duration,self.label]],
                                 columns=['mean_smv', 'max_smv','min_smv','duration','label'])
        return newLine
        

In [9]:
# Test Window class
event = 'CSI'
FilePath = os.path.join(os.getcwd(), 'MobiAct/{}/{}_1_1_annotated.csv'.format(event,event))
wins_test = Window.getWindows(FilePath, event, 200, 0.8, 5)
# wins_test[0].visualize()
# wins_test[1].visualize()
print(len(wins_test))
print(wins_test[0].export())
print(wins_test[1].export())

2
   mean_smv    max_smv   min_smv  duration  label
0  9.928736  18.714399  3.281083        13      0
   mean_smv    max_smv   min_smv  duration  label
0  9.874734  18.714399  3.281083        13      0


In [10]:
ADLs = {'STD':1,'WAL':1,'JOG':3,'JUM':3,'STU':6,'STN':6,'SCH':6,'SIT':1,'CHU':3,"CSI":3,'CSO':3,'LYI':0}
Falls = {'FOL':3,'FKL':3,'BSC':3,'SDL':3}
Events = ['STD','WAL','JOG','JUM','STU','STN','SCH','SIT','CHU',"CSI",'CSO','LYI','FOL','FKL','BSC','SDL']
# participants number
pNum = 67
# sampling rate (Hz)
sampleRate = 200
# overlap rate between window is 0.8 
# i.e. in the case setting windows size = 5s, the interval between windows is 1s 
overlapRate = 0.8
# window size = 5s
wSize = 5

In [11]:
dataset = pd.DataFrame(columns=['mean_smv','max_smv','min_smv','duration','label'])
for event in Events:
    if event in ADLs.keys():
        trialTimes = ADLs.get(event)
    else:
        trialTimes = Falls.get(event)
    for i in range(1, pNum + 1):
        for j in range(1, trialTimes + 1):     
            fileName = 'MobiAct/{}/{}_{}_{}_annotated.csv'.format(event, event, i, j)
            dataPath = os.path.join(os.getcwd(), fileName)
            try:
                windows = Window.getWindows(dataPath, event, sampleRate, overlapRate, wSize)
                print(fileName)
                for window in windows:
                    newWindow = window.export()
                    dataset = pd.concat([dataset, newWindow], ignore_index=True)
            except FileNotFoundError:
                print('{} is not found'.format(fileName))
            continue
 

MobiAct/STD/STD_1_1_annotated.csv
MobiAct/STD/STD_2_1_annotated.csv
MobiAct/STD/STD_3_1_annotated.csv
MobiAct/STD/STD_4_1_annotated.csv
MobiAct/STD/STD_5_1_annotated.csv
MobiAct/STD/STD_6_1_annotated.csv
MobiAct/STD/STD_7_1_annotated.csv
MobiAct/STD/STD_8_1_annotated.csv
MobiAct/STD/STD_9_1_annotated.csv
MobiAct/STD/STD_10_1_annotated.csv
MobiAct/STD/STD_11_1_annotated.csv
MobiAct/STD/STD_12_1_annotated.csv
MobiAct/STD/STD_13_1_annotated.csv is not found
MobiAct/STD/STD_14_1_annotated.csv is not found
MobiAct/STD/STD_15_1_annotated.csv is not found
MobiAct/STD/STD_16_1_annotated.csv
MobiAct/STD/STD_17_1_annotated.csv is not found
MobiAct/STD/STD_18_1_annotated.csv
MobiAct/STD/STD_19_1_annotated.csv
MobiAct/STD/STD_20_1_annotated.csv
MobiAct/STD/STD_21_1_annotated.csv
MobiAct/STD/STD_22_1_annotated.csv
MobiAct/STD/STD_23_1_annotated.csv
MobiAct/STD/STD_24_1_annotated.csv
MobiAct/STD/STD_25_1_annotated.csv
MobiAct/STD/STD_26_1_annotated.csv
MobiAct/STD/STD_27_1_annotated.csv
MobiAct/STD/

MobiAct/JOG/JOG_32_1_annotated.csv
MobiAct/JOG/JOG_32_2_annotated.csv
MobiAct/JOG/JOG_32_3_annotated.csv
MobiAct/JOG/JOG_33_1_annotated.csv
MobiAct/JOG/JOG_33_2_annotated.csv
MobiAct/JOG/JOG_33_3_annotated.csv
MobiAct/JOG/JOG_34_1_annotated.csv
MobiAct/JOG/JOG_34_2_annotated.csv
MobiAct/JOG/JOG_34_3_annotated.csv
MobiAct/JOG/JOG_35_1_annotated.csv
MobiAct/JOG/JOG_35_2_annotated.csv
MobiAct/JOG/JOG_35_3_annotated.csv
MobiAct/JOG/JOG_36_1_annotated.csv
MobiAct/JOG/JOG_36_2_annotated.csv
MobiAct/JOG/JOG_36_3_annotated.csv
MobiAct/JOG/JOG_37_1_annotated.csv
MobiAct/JOG/JOG_37_2_annotated.csv
MobiAct/JOG/JOG_37_3_annotated.csv
MobiAct/JOG/JOG_38_1_annotated.csv
MobiAct/JOG/JOG_38_2_annotated.csv
MobiAct/JOG/JOG_38_3_annotated.csv
MobiAct/JOG/JOG_39_1_annotated.csv
MobiAct/JOG/JOG_39_2_annotated.csv
MobiAct/JOG/JOG_39_3_annotated.csv
MobiAct/JOG/JOG_40_1_annotated.csv
MobiAct/JOG/JOG_40_2_annotated.csv
MobiAct/JOG/JOG_40_3_annotated.csv
MobiAct/JOG/JOG_41_1_annotated.csv
MobiAct/JOG/JOG_41_2

MobiAct/JUM/JUM_41_2_annotated.csv
MobiAct/JUM/JUM_41_3_annotated.csv
MobiAct/JUM/JUM_42_1_annotated.csv
MobiAct/JUM/JUM_42_2_annotated.csv
MobiAct/JUM/JUM_42_3_annotated.csv
MobiAct/JUM/JUM_43_1_annotated.csv
MobiAct/JUM/JUM_43_2_annotated.csv
MobiAct/JUM/JUM_43_3_annotated.csv
MobiAct/JUM/JUM_44_1_annotated.csv
MobiAct/JUM/JUM_44_2_annotated.csv
MobiAct/JUM/JUM_44_3_annotated.csv
MobiAct/JUM/JUM_45_1_annotated.csv
MobiAct/JUM/JUM_45_2_annotated.csv
MobiAct/JUM/JUM_45_3_annotated.csv
MobiAct/JUM/JUM_46_1_annotated.csv
MobiAct/JUM/JUM_46_2_annotated.csv
MobiAct/JUM/JUM_46_3_annotated.csv
MobiAct/JUM/JUM_47_1_annotated.csv
MobiAct/JUM/JUM_47_2_annotated.csv
MobiAct/JUM/JUM_47_3_annotated.csv
MobiAct/JUM/JUM_48_1_annotated.csv
MobiAct/JUM/JUM_48_2_annotated.csv
MobiAct/JUM/JUM_48_3_annotated.csv
MobiAct/JUM/JUM_49_1_annotated.csv
MobiAct/JUM/JUM_49_2_annotated.csv
MobiAct/JUM/JUM_49_3_annotated.csv
MobiAct/JUM/JUM_50_1_annotated.csv
MobiAct/JUM/JUM_50_2_annotated.csv
MobiAct/JUM/JUM_50_3

MobiAct/STU/STU_25_6_annotated.csv
MobiAct/STU/STU_26_1_annotated.csv
MobiAct/STU/STU_26_2_annotated.csv
MobiAct/STU/STU_26_3_annotated.csv
MobiAct/STU/STU_26_4_annotated.csv
MobiAct/STU/STU_26_5_annotated.csv
MobiAct/STU/STU_26_6_annotated.csv
MobiAct/STU/STU_27_1_annotated.csv
MobiAct/STU/STU_27_2_annotated.csv
MobiAct/STU/STU_27_3_annotated.csv is not found
MobiAct/STU/STU_27_4_annotated.csv
MobiAct/STU/STU_27_5_annotated.csv
MobiAct/STU/STU_27_6_annotated.csv
MobiAct/STU/STU_28_1_annotated.csv
MobiAct/STU/STU_28_2_annotated.csv
MobiAct/STU/STU_28_3_annotated.csv
MobiAct/STU/STU_28_4_annotated.csv
MobiAct/STU/STU_28_5_annotated.csv
MobiAct/STU/STU_28_6_annotated.csv
MobiAct/STU/STU_29_1_annotated.csv
MobiAct/STU/STU_29_2_annotated.csv
MobiAct/STU/STU_29_3_annotated.csv
MobiAct/STU/STU_29_4_annotated.csv
MobiAct/STU/STU_29_5_annotated.csv
MobiAct/STU/STU_29_6_annotated.csv
MobiAct/STU/STU_30_1_annotated.csv is not found
MobiAct/STU/STU_30_2_annotated.csv is not found
MobiAct/STU/STU_

MobiAct/STU/STU_64_4_annotated.csv
MobiAct/STU/STU_64_5_annotated.csv
MobiAct/STU/STU_64_6_annotated.csv
MobiAct/STU/STU_65_1_annotated.csv
MobiAct/STU/STU_65_2_annotated.csv
MobiAct/STU/STU_65_3_annotated.csv
MobiAct/STU/STU_65_4_annotated.csv
MobiAct/STU/STU_65_5_annotated.csv
MobiAct/STU/STU_65_6_annotated.csv
MobiAct/STU/STU_66_1_annotated.csv
MobiAct/STU/STU_66_2_annotated.csv
MobiAct/STU/STU_66_3_annotated.csv
MobiAct/STU/STU_66_4_annotated.csv
MobiAct/STU/STU_66_5_annotated.csv
MobiAct/STU/STU_66_6_annotated.csv
MobiAct/STU/STU_67_1_annotated.csv
MobiAct/STU/STU_67_2_annotated.csv
MobiAct/STU/STU_67_3_annotated.csv
MobiAct/STU/STU_67_4_annotated.csv
MobiAct/STU/STU_67_5_annotated.csv
MobiAct/STU/STU_67_6_annotated.csv
MobiAct/STN/STN_1_1_annotated.csv
MobiAct/STN/STN_1_2_annotated.csv
MobiAct/STN/STN_1_3_annotated.csv
MobiAct/STN/STN_1_4_annotated.csv
MobiAct/STN/STN_1_5_annotated.csv
MobiAct/STN/STN_1_6_annotated.csv
MobiAct/STN/STN_2_1_annotated.csv
MobiAct/STN/STN_2_2_annotat

...

MobiAct/CSO/CSO_11_2_annotated.csv
MobiAct/CSO/CSO_11_3_annotated.csv
MobiAct/CSO/CSO_12_1_annotated.csv
MobiAct/CSO/CSO_12_2_annotated.csv
MobiAct/CSO/CSO_12_3_annotated.csv
MobiAct/CSO/CSO_13_1_annotated.csv is not found
MobiAct/CSO/CSO_13_2_annotated.csv is not found
MobiAct/CSO/CSO_13_3_annotated.csv is not found
MobiAct/CSO/CSO_14_1_annotated.csv is not found
MobiAct/CSO/CSO_14_2_annotated.csv is not found
MobiAct/CSO/CSO_14_3_annotated.csv is not found
MobiAct/CSO/CSO_15_1_annotated.csv is not found
MobiAct/CSO/CSO_15_2_annotated.csv is not found
MobiAct/CSO/CSO_15_3_annotated.csv is not found
MobiAct/CSO/CSO_16_1_annotated.csv
MobiAct/CSO/CSO_16_2_annotated.csv
MobiAct/CSO/CSO_16_3_annotated.csv
MobiAct/CSO/CSO_17_1_annotated.csv is not found
MobiAct/CSO/CSO_17_2_annotated.csv is not found
MobiAct/CSO/CSO_17_3_annotated.csv is not found
MobiAct/CSO/CSO_18_1_annotated.csv
MobiAct/CSO/CSO_18_2_annotated.csv
MobiAct/CSO/CSO_18_3_annotated.csv
MobiAct/CSO/CSO_19_1_annotated.csv
Mobi

MobiAct/FOL/FOL_20_3_annotated.csv
MobiAct/FOL/FOL_21_1_annotated.csv
MobiAct/FOL/FOL_21_2_annotated.csv
MobiAct/FOL/FOL_21_3_annotated.csv
MobiAct/FOL/FOL_22_1_annotated.csv
MobiAct/FOL/FOL_22_2_annotated.csv
MobiAct/FOL/FOL_22_3_annotated.csv
MobiAct/FOL/FOL_23_1_annotated.csv
MobiAct/FOL/FOL_23_2_annotated.csv
MobiAct/FOL/FOL_23_3_annotated.csv
MobiAct/FOL/FOL_24_1_annotated.csv is not found
MobiAct/FOL/FOL_24_2_annotated.csv is not found
MobiAct/FOL/FOL_24_3_annotated.csv is not found
MobiAct/FOL/FOL_25_1_annotated.csv
MobiAct/FOL/FOL_25_2_annotated.csv
MobiAct/FOL/FOL_25_3_annotated.csv
MobiAct/FOL/FOL_26_1_annotated.csv
MobiAct/FOL/FOL_26_2_annotated.csv
MobiAct/FOL/FOL_26_3_annotated.csv
MobiAct/FOL/FOL_27_1_annotated.csv
MobiAct/FOL/FOL_27_2_annotated.csv
MobiAct/FOL/FOL_27_3_annotated.csv
MobiAct/FOL/FOL_28_1_annotated.csv
MobiAct/FOL/FOL_28_2_annotated.csv
MobiAct/FOL/FOL_28_3_annotated.csv
MobiAct/FOL/FOL_29_1_annotated.csv
MobiAct/FOL/FOL_29_2_annotated.csv
MobiAct/FOL/FOL_

MobiAct/FKL/FKL_30_3_annotated.csv
MobiAct/FKL/FKL_31_1_annotated.csv
MobiAct/FKL/FKL_31_2_annotated.csv
MobiAct/FKL/FKL_31_3_annotated.csv
MobiAct/FKL/FKL_32_1_annotated.csv
MobiAct/FKL/FKL_32_2_annotated.csv
MobiAct/FKL/FKL_32_3_annotated.csv
MobiAct/FKL/FKL_33_1_annotated.csv
MobiAct/FKL/FKL_33_2_annotated.csv
MobiAct/FKL/FKL_33_3_annotated.csv
MobiAct/FKL/FKL_34_1_annotated.csv
MobiAct/FKL/FKL_34_2_annotated.csv
MobiAct/FKL/FKL_34_3_annotated.csv
MobiAct/FKL/FKL_35_1_annotated.csv
MobiAct/FKL/FKL_35_2_annotated.csv
MobiAct/FKL/FKL_35_3_annotated.csv
MobiAct/FKL/FKL_36_1_annotated.csv
MobiAct/FKL/FKL_36_2_annotated.csv
MobiAct/FKL/FKL_36_3_annotated.csv
MobiAct/FKL/FKL_37_1_annotated.csv
MobiAct/FKL/FKL_37_2_annotated.csv
MobiAct/FKL/FKL_37_3_annotated.csv
MobiAct/FKL/FKL_38_1_annotated.csv
MobiAct/FKL/FKL_38_2_annotated.csv
MobiAct/FKL/FKL_38_3_annotated.csv
MobiAct/FKL/FKL_39_1_annotated.csv
MobiAct/FKL/FKL_39_2_annotated.csv
MobiAct/FKL/FKL_39_3_annotated.csv
MobiAct/FKL/FKL_40_1

MobiAct/BSC/BSC_42_1_annotated.csv
MobiAct/BSC/BSC_42_2_annotated.csv
MobiAct/BSC/BSC_42_3_annotated.csv
MobiAct/BSC/BSC_43_1_annotated.csv
MobiAct/BSC/BSC_43_2_annotated.csv
MobiAct/BSC/BSC_43_3_annotated.csv
MobiAct/BSC/BSC_44_1_annotated.csv
MobiAct/BSC/BSC_44_2_annotated.csv
MobiAct/BSC/BSC_44_3_annotated.csv
MobiAct/BSC/BSC_45_1_annotated.csv
MobiAct/BSC/BSC_45_2_annotated.csv
MobiAct/BSC/BSC_45_3_annotated.csv
MobiAct/BSC/BSC_46_1_annotated.csv
MobiAct/BSC/BSC_46_2_annotated.csv
MobiAct/BSC/BSC_46_3_annotated.csv
MobiAct/BSC/BSC_47_1_annotated.csv
MobiAct/BSC/BSC_47_2_annotated.csv
MobiAct/BSC/BSC_47_3_annotated.csv
MobiAct/BSC/BSC_48_1_annotated.csv
MobiAct/BSC/BSC_48_2_annotated.csv
MobiAct/BSC/BSC_48_3_annotated.csv
MobiAct/BSC/BSC_49_1_annotated.csv
MobiAct/BSC/BSC_49_2_annotated.csv
MobiAct/BSC/BSC_49_3_annotated.csv
MobiAct/BSC/BSC_50_1_annotated.csv is not found
MobiAct/BSC/BSC_50_2_annotated.csv is not found
MobiAct/BSC/BSC_50_3_annotated.csv is not found
MobiAct/BSC/BSC_

MobiAct/SDL/SDL_52_1_annotated.csv
MobiAct/SDL/SDL_52_2_annotated.csv
MobiAct/SDL/SDL_52_3_annotated.csv
MobiAct/SDL/SDL_53_1_annotated.csv
MobiAct/SDL/SDL_53_2_annotated.csv
MobiAct/SDL/SDL_53_3_annotated.csv
MobiAct/SDL/SDL_54_1_annotated.csv
MobiAct/SDL/SDL_54_2_annotated.csv
MobiAct/SDL/SDL_54_3_annotated.csv
MobiAct/SDL/SDL_55_1_annotated.csv
MobiAct/SDL/SDL_55_2_annotated.csv
MobiAct/SDL/SDL_55_3_annotated.csv
MobiAct/SDL/SDL_56_1_annotated.csv
MobiAct/SDL/SDL_56_2_annotated.csv
MobiAct/SDL/SDL_56_3_annotated.csv
MobiAct/SDL/SDL_57_1_annotated.csv
MobiAct/SDL/SDL_57_2_annotated.csv
MobiAct/SDL/SDL_57_3_annotated.csv
MobiAct/SDL/SDL_58_1_annotated.csv
MobiAct/SDL/SDL_58_2_annotated.csv
MobiAct/SDL/SDL_58_3_annotated.csv
MobiAct/SDL/SDL_59_1_annotated.csv
MobiAct/SDL/SDL_59_2_annotated.csv
MobiAct/SDL/SDL_59_3_annotated.csv
MobiAct/SDL/SDL_60_1_annotated.csv
MobiAct/SDL/SDL_60_2_annotated.csv
MobiAct/SDL/SDL_60_3_annotated.csv
MobiAct/SDL/SDL_61_1_annotated.csv
MobiAct/SDL/SDL_61_2

In [12]:
SavePath = os.path.join(os.getcwd(), 'Windows_Data/dataset_4f.csv')
dataset.to_csv(SavePath)
dataset

,mean_smv,max_smv,min_smv,duration,label
0,9.771294,9.834680,9.693022,526,0
1,9.773626,9.834680,9.693022,526,0
2,9.772474,9.832309,9.693022,429,0
3,9.774087,9.813454,9.693022,-698,0
4,9.775211,9.813841,9.693022,-863,0
...,...,...,...,...,...
56912,10.021342,30.279845,4.210766,52,1
56913,9.708413,15.671375,5.639311,-11,0
56914,9.623634,9.926912,9.240420,31,0
56915,9.624513,9.911831,9.358716,-10,0
